Dataset : https://www.kaggle.com/competitions/titanic  
Another dataset to explore : https://www.kaggle.com/competitions?sortOption=numTeams&hostSegmentIdFilter=5

Objective : provide prediction of who will survive and who wont on test dataset and submit the result

In [1]:
import os 
import pandas as pd
import seaborn as sns
import plotly.express as px

In [2]:
data_path = os.getcwd() + '\\titanic-data\\train.csv'
base_df = pd.read_csv(data_path)

In [3]:
# There's many null column especially in Cabin Column
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
df = base_df.drop(columns='Cabin')

# Explore the Data

## See Each Column

In [5]:
# bagaimana rasio umur yang onboard 
# rasio umur
# Sibsp
# Parch
# Ticket
# Fare
# Embarked
# dst

In [15]:
survive_bar = df.groupby('Survived').count()[['PassengerId']].reset_index()
survive_bar['persen'] = round((survive_bar['PassengerId'] / survive_bar['PassengerId'].sum() * 100),2)
survive_bar

,Survived,PassengerId,persen
0,0,549,61.62
1,1,342,38.38


In [16]:
px.bar(data_frame=survive_bar, x='Survived', y='PassengerId', text=survive_bar['persen'], title='Survived Status on Ship')

In [6]:
bar_sex = df.groupby('Sex').count()[['PassengerId']].reset_index()
bar_sex['persen'] = round((bar_sex['PassengerId'] / bar_sex['PassengerId'].sum() * 100),2)
bar_sex

,Sex,PassengerId,persen
0,female,314,35.24
1,male,577,64.76


In [7]:
# Apakah yang male ini pada nggak selamat?
px.bar(data_frame=bar_sex, x='Sex', y='PassengerId', text=bar_sex['persen'], title='Sexes on Ship')

In [8]:
# Usia paling banyak di antara tahun 20 - 30 tahun
px.histogram(df['Age'], text_auto=True, title='Age on Ship')

In [31]:
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [12]:
sibs_bar = df.groupby('SibSp').count()[['PassengerId']].reset_index()
sibs_bar['persen'] = round((sibs_bar['PassengerId'] / sibs_bar['PassengerId'].sum() * 100),2)
sibs_bar

,SibSp,PassengerId,persen
0,0,608,68.24
1,1,209,23.46
2,2,28,3.14
3,3,16,1.80
4,4,18,2.02
5,5,5,0.56
6,8,7,0.79


In [14]:
# Mungkin secara logika bisa aja kalo punya siblings dia bakal lebih mungkin selamat
# Persenan selamat & siblingsnya berbeda terbalik
# Dan datanya skewed, bisa aja gak ngaruh
px.bar(data_frame=sibs_bar, x='SibSp', y='PassengerId', text=sibs_bar['persen'], title='Have Siblings on Ship')

In [18]:
# pclass: A proxy for socio-economic status (SES) {1:upper, 2:middle, 3:lower}
pclass_bar = df.groupby('Pclass').count()[['PassengerId']].reset_index()
pclass_bar['persen'] = round((pclass_bar['PassengerId'] / pclass_bar['PassengerId'].sum() * 100),2)
pclass_bar

,Pclass,PassengerId,persen
0,1,216,24.24
1,2,184,20.65
2,3,491,55.11


In [19]:
px.bar(data_frame=pclass_bar, x='Pclass', y='PassengerId', text=pclass_bar['persen'], title='Pclass on Ship')

In [22]:
# Parch : Hubungan dengan saudara
""" parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.
"""

parch_bar = df.groupby('Parch').count()[['PassengerId']].reset_index()
parch_bar['persen'] = round((parch_bar['PassengerId'] / parch_bar['PassengerId'].sum() * 100),2)
parch_bar

,Parch,PassengerId,persen
0,0,678,76.09
1,1,118,13.24
2,2,80,8.98
3,3,5,0.56
4,4,4,0.45
5,5,5,0.56
6,6,1,0.11


In [25]:
# sedikit nggak jelas pengertian parch ini
px.bar(data_frame=parch_bar, x='Parch', y='PassengerId', text=parch_bar['persen'], title='Parch on Ship')

In [27]:
# Unique Value kebanyakan, perlu analisa lebih lanjut / di drop aja
ticket_bar = df.groupby('Ticket').count()[['PassengerId']].reset_index()
ticket_bar['persen'] = round((ticket_bar['PassengerId'] / ticket_bar['PassengerId'].sum() * 100),2)
ticket_bar

,Ticket,PassengerId,persen
0,110152,3,0.34
1,110413,3,0.34
2,110465,2,0.22
3,110564,1,0.11
4,110813,1,0.11
...,...,...,...
676,W./C. 6608,4,0.45
677,W./C. 6609,1,0.11
678,W.E.P. 5734,1,0.11
679,W/C 14208,1,0.11


In [29]:
# Fare kebanyakan, kemungkinan perlu di binning. Ada yang Outlier juga
px.histogram(df['Fare'], text_auto=True, title='Fare on Ship')

In [30]:
# Kemungkinan binningnya berdasarkan Quantile ini
df['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [32]:
# Port of Embarkation
embarked_bar = df.groupby('Embarked').count()[['PassengerId']].reset_index()
embarked_bar['persen'] = round((embarked_bar['PassengerId'] / embarked_bar['PassengerId'].sum() * 100),2)
embarked_bar

,Embarked,PassengerId,persen
0,C,168,18.90
1,Q,77,8.66
2,S,644,72.44


In [33]:
px.bar(data_frame=embarked_bar, x='Embarked', y='PassengerId', text=embarked_bar['persen'], title='Parch on Ship')

In [34]:
# Drop Column Ticket, too much diversity
df1 = df.drop(columns='Ticket')

## Column in relation with survival

In [35]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Fare         891 non-null    float64
 9   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


### Survived vs Embarked

In [53]:
df_sus_embrk = df[['Survived', 'Embarked', 'PassengerId']].groupby(['Survived', 'Embarked']).count().reset_index()
df_sus_embrk['persen'] = df_sus_embrk['PassengerId'] / df_sus_embrk['PassengerId'].sum() * 100
df_sus_embrk

,Survived,Embarked,PassengerId,persen
0,0,C,75,8.436445
1,0,Q,47,5.286839
2,0,S,427,48.031496
3,1,C,93,10.461192
4,1,Q,30,3.374578
5,1,S,217,24.409449


In [58]:
px.histogram(df_sus_embrk, x="Embarked", y="PassengerId", color='Survived', barmode='group', text_auto=True, title="Survived vs Embarked")

In [ ]:
# persenan yang ini kan dari total, gimana caranya nambahin persenan tapi per group biar lebih keliatan perbedaannya
# gimana caranya nambahin histogram ke px histogram
# Analisa untuk kolom sisanya